In [1]:
from datetime import datetime
import pandas as pd
from selenium import webdriver


week_str = '20221113'
# datetime object containing current date and time
now = datetime.now()


d_id_to_city = {'1078741': 'Chicago, IL', 
                'RunDMRC': 'Dallas, TX', 
                'lamrc': 'Los Angeles, CA',
                'nycmrc': 'New York, NY',
                '1020331': 'San Francisco, CA',
                'detroitmrc': 'Detroit, MI',
                'dcmuslimsrun': 'Washington, DC',
                '1088734': 'Houston, TX',
                'seattlemrc': 'Seattle, WA',
                'sdmrc': 'San Diego, CA',
                }

d_club_data = {}
for club in d_id_to_city:
    driver = webdriver.Chrome()
    location = d_id_to_city[club]
    driver.get('https://www.strava.com/clubs/' + club + '/leaderboard')
    
    dfs = pd.read_html(driver.page_source)
    
    d_club_data[location] = dfs[-1]
    print(location, len(dfs[-1]))
    d_club_data[location]['timestamp'] = now
    d_club_data[location]['location'] = location
    
    driver.close()


Chicago, IL 18
Dallas, TX 41
Los Angeles, CA 14
New York, NY 29
San Francisco, CA 31
Detroit, MI 6
Washington, DC 6
Houston, TX 3
Seattle, WA 4
San Diego, CA 2


In [2]:
from math import floor

# handle LA

def fixtime(t):
    if 'h' in t[0]:
        return (float(t[0][:-1]) * 60) + float(t[1][:-1])
    return float(t[0][:-1])

d_club_data['Los Angeles, CA']['time_hold'] = d_club_data['Los Angeles, CA']['Time'].map(lambda x: x.split(" "))
d_club_data['Los Angeles, CA']['time_hold'] = d_club_data['Los Angeles, CA']['time_hold'].map(lambda x: fixtime(x))
d_club_data['Los Angeles, CA']['dist_LA'] = d_club_data['Los Angeles, CA']['Distance'].map(lambda x: float(x[:-3]))
d_club_data['Los Angeles, CA']['Avg. Pace'] = d_club_data['Los Angeles, CA']['time_hold'] / d_club_data['Los Angeles, CA']['dist_LA']
d_club_data['Los Angeles, CA']['Avg. Pace'] = d_club_data['Los Angeles, CA']['Avg. Pace'].map(lambda x: str(floor(x)) + ":" + str(int((x - floor(x)) * 60)) + " /km")
d_club_data['Los Angeles, CA']

,Rank,Athlete,Distance,Activities,Elev. Gain,Time,timestamp,location,time_hold,dist_LA,Avg. Pace
0,1,Didi R.,27.2 km,3,94 m,2h 30m,2022-11-12 19:28:18.006347,"Los Angeles, CA",150.0,27.2,5:30 /km
1,2,shajee’ah 🐿.,24.4 km,5,107 m,3h 1m,2022-11-12 19:28:18.006347,"Los Angeles, CA",181.0,24.4,7:25 /km
2,3,Sanaa O.,20.5 km,4,259 m,2h 10m,2022-11-12 19:28:18.006347,"Los Angeles, CA",130.0,20.5,6:20 /km
3,4,Ayesha H.,20.3 km,3,376 m,3h 12m,2022-11-12 19:28:18.006347,"Los Angeles, CA",192.0,20.3,9:27 /km
4,5,Ahmed A.,15.9 km,4,--,2h 33m,2022-11-12 19:28:18.006347,"Los Angeles, CA",153.0,15.9,9:37 /km
5,6,Meagan B.,13.1 km,4,68 m,1h 31m,2022-11-12 19:28:18.006347,"Los Angeles, CA",91.0,13.1,6:56 /km
6,7,Cyrus P.,10.6 km,2,79 m,1h 12m,2022-11-12 19:28:18.006347,"Los Angeles, CA",72.0,10.6,6:47 /km
7,8,Nida A.,9.6 km,1,68 m,1h 9m,2022-11-12 19:28:18.006347,"Los Angeles, CA",69.0,9.6,7:11 /km
8,9,Sohaib N.,6.7 km,1,--,35m,2022-11-12 19:28:18.006347,"Los Angeles, CA",35.0,6.7,5:13 /km
9,10,Ahhad I.,6.3 km,1,28 m,46m,2022-11-12 19:28:18.006347,"Los Angeles, CA",46.0,6.3,7:18 /km


In [3]:
df_all = pd.concat([d_club_data[k] for k in d_club_data])
df_all = df_all[df_all['Distance'] != 'There are no results.']
df_all['dist_float'] = df_all['Distance'].map(lambda x: float(x[:-3]))
df_all['avg_pace_float'] = df_all['Avg. Pace'].map(lambda x: x[:-4].split(":"))
df_all['avg_pace_km_min'] = df_all['avg_pace_float'].map(lambda x: float((60 * int(x[0])) + int(x[1])) / 60)
df_all['time_spent'] = df_all['avg_pace_km_min'] * df_all['dist_float']

df_all = df_all.sort_values(by=['dist_float'], ascending=False)
df_all['National Rank'] = [x for x in range(1, len(df_all) + 1)]
df_all['Location Rank'] = df_all['Rank']
df_all['elevation_gain'] = df_all['Elev. Gain']
df_all['elevation_gain'] = df_all['elevation_gain'].map(lambda x: ''.join(x.split(",")))
df_all['elevation_gain'] = df_all['elevation_gain'].map(lambda x: float(x[:-2]) if x != '--' else 0)
df_all.to_csv("rawdata/" + "raw_data_" + week_str + ".csv", index=False)

df_fts = df_all[['Athlete', 'dist_float', 'avg_pace_km_min', 'elevation_gain', 'time_spent', 'location']].copy(deep=True)

# dist rank
df_fts = df_fts.sort_values(by=['dist_float'], ascending=False)
df_fts['dist_rank'] = [x for x in range(1, len(df_fts) + 1)]

# avg_pace_km_min rank
df_fts = df_fts.sort_values(by=['avg_pace_km_min'], ascending=True)
df_fts['pace_rank'] = [x for x in range(1, len(df_fts) + 1)]

# elevation_gain rank
df_fts = df_fts.sort_values(by=['elevation_gain'], ascending=False)
df_fts['elevation_rank'] = [x for x in range(1, len(df_fts) + 1)]

df_all = df_all[['National Rank', 'Athlete', 'Distance', 'Runs', 'Longest', 'elevation_gain', 'timestamp', 'location', 'dist_float', 'Location Rank']]

In [4]:
df_all.sample(5)

,National Rank,Athlete,Distance,Runs,Longest,elevation_gain,timestamp,location,dist_float,Location Rank
8,49,Mahaa M.,16.3 km,5.0,6.7 km,26.0,2022-11-12 19:28:18.006347,"San Francisco, CA",16.3,9
27,104,Neelam Z.,5.8 km,2.0,3.5 km,10.0,2022-11-12 19:28:18.006347,"Dallas, TX",5.8,28
26,144,Zuhair K.,3.0 km,1.0,3.0 km,22.0,2022-11-12 19:28:18.006347,"New York, NY",3.0,27
1,28,shajee’ah 🐿.,24.4 km,NaN,NaN,107.0,2022-11-12 19:28:18.006347,"Los Angeles, CA",24.4,2
15,69,Tehreem S.,9.9 km,2.0,4.9 km,37.0,2022-11-12 19:28:18.006347,"Dallas, TX",9.9,16


In [5]:
df_fts['National Rank'] = (df_fts['dist_rank'] + df_fts['pace_rank'] + df_fts['elevation_rank']) / 3
df_fts = df_fts.sort_values(by=['National Rank'], ascending=True)
df_fts['National Rank'] = [x for x in range(1, len(df_fts) + 1)]
df_fts = df_fts[['National Rank', 'Athlete', 'dist_float', 'avg_pace_km_min', 'elevation_gain', 'time_spent', 'location']]
df_fts

,National Rank,Athlete,dist_float,avg_pace_km_min,elevation_gain,time_spent,location
0,1,Samir R.,55.6,4.633333,182.0,257.613333,"San Francisco, CA"
0,2,Shehryar K.,78.7,4.900000,264.0,385.630000,"New York, NY"
0,3,Nour A.,49.8,5.283333,1257.0,263.110000,"Dallas, TX"
1,4,Engr Khurram S.,46.5,5.433333,534.0,252.650000,"San Francisco, CA"
4,5,Sufyan K.,40.2,4.800000,79.0,192.960000,"Chicago, IL"
...,...,...,...,...,...,...,...
33,150,Sadia N.,4.2,8.583333,0.0,36.050000,"Dallas, TX"
25,151,Mariam B.,3.7,7.916667,0.0,29.291667,"New York, NY"
34,152,Mohammed Mustafa S.,3.6,7.933333,0.0,28.560000,"Dallas, TX"
28,153,Ebrahim A.,3.5,7.800000,0.0,27.300000,"San Francisco, CA"


In [6]:
df_fts.groupby(['location']).sum('dist_float')

,National Rank,dist_float,avg_pace_km_min,elevation_gain,time_spent
location,,,,,
"Chicago, IL",1373,360.0,112.783333,403.0,2112.680000
"Dallas, TX",3385,583.2,284.783333,3544.0,3790.021667
"Detroit, MI",429,80.5,36.716667,282.0,447.743333
"Houston, TX",275,59.6,22.200000,93.0,428.800000
"Los Angeles, CA",1051,171.8,112.616667,1222.0,1280.601667
"New York, NY",2124,409.6,186.850000,1652.0,2576.865000
"San Diego, CA",103,33.1,12.750000,105.0,208.145000
"San Francisco, CA",2541,409.6,193.366667,1832.0,2472.381667
"Seattle, WA",207,39.1,23.266667,556.0,227.106667


In [7]:
df_fts.to_csv('MRC_week_' + week_str + '.csv', index=False)

In [8]:
df_all.groupby(['location']).sum('dist_float')

,National Rank,Runs,elevation_gain,dist_float,Location Rank
location,,,,,
"Chicago, IL",1263,54.0,403.0,360.0,171
"Dallas, TX",3257,94.0,3544.0,583.2,861
"Detroit, MI",482,16.0,282.0,80.5,21
"Houston, TX",209,6.0,93.0,59.6,6
"Los Angeles, CA",1069,0.0,1222.0,171.8,105
"New York, NY",2329,64.0,1652.0,409.6,435
"San Diego, CA",101,5.0,105.0,33.1,3
"San Francisco, CA",2429,69.0,1832.0,409.6,496
"Seattle, WA",369,7.0,556.0,39.1,10
